#Paquetes necesarios

In [ ]:
import cv2  
import math 

from ultralytics import YOLO

Entrenamiento del modelo que detecta matrículas de coches

In [ ]:
import cv2  
import math 

from ultralytics import YOLO

Prueba del modelo creado

In [ ]:
model = YOLO('runs/detect/train9/weights/best.pt')  #Aqui te descargas el modelo best.pt de la run

#Para un vídeo 
filename = "C0142.MP4"
results = model(filename, show=True)

cv2.destroyAllWindows()

Uso de nuestro modelo combinado con el de personas

In [2]:
import cv2
from ultralytics import YOLO

# Load the models
model1 = YOLO('yolo11n-seg.pt')  # Model for segmentation (masks)
model2 = YOLO('runs/detect/train9/weights/best.pt')  # Model for pose estimation

# Define the video source
filename = "C0142.MP4"
cap = cv2.VideoCapture(filename)

# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference with the first model (segmentation)
    results1 = model1.predict(source=frame, augment=True, show=False)
    
    # Perform inference with the second model (pose estimation)
    results2 = model2.predict(source=frame, augment=True, show=False)

    # Visualize results from the first model (masks)
    annotated_frame1 = results1[0].plot()  # Draw segmentation results

    # Visualize results from the second model (pose estimation)
    annotated_frame2 = results2[0].plot()  # Draw pose results

    # Combine the annotations from both models
    combined_frame = cv2.addWeighted(annotated_frame1, 0.5, annotated_frame2, 0.5, 0)

    # Display the combined results
    cv2.imshow('Combined Results', combined_frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

AttributeError: module 'torch' has no attribute '__version__'